In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass     #this goes in the entity file
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [4]:
import sys
import os

sys.path.append(os.path.abspath("src"))

from Briefly.constants import *  # Now it should work
from Briefly.utils.common import  read_yaml, create_directories
# this goes in configuration

In [5]:
class ConfigurationManager:           #This goes into configuration.py
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        """Initialize Configuration Manager and load YAML configs."""
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure artifacts directory exists
        create_directories([self.config["artifacts_root"]])   #this part is same in ingestion and validation
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config
        
        
        

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

C:\Users\sheet\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-20 17:03:41,980: WARNING: From c:\Users\sheet\.conda\envs\textSum\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2025-06-20 17:03:43,629: INFO: PyTorch version 2.5.1 available.]
[2025-06-20 17:03:43,640: INFO: TensorFlow version 2.19.0 available.]
[2025-06-20 17:03:43,645: INFO: JAX version 0.5.1 available.]


In [7]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        # Load preprocessed dataset (already tokenized during DataTransformation)
        dataset = load_from_disk(self.config.data_path)

        # Data collator
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        # Training arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=16
        )

        # Trainer
        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"]
        )

        # Start training
        trainer.train()

        # Save model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-06-20 17:03:46,503: INFO: yaml file: config\config.yaml loaded successfully]
[2025-06-20 17:03:46,511: INFO: yaml file: params.yaml loaded successfully]
[2025-06-20 17:03:46,518: INFO: created directory at: artifacts]
[2025-06-20 17:03:46,522: INFO: created directory at: artifacts/model_trainer]


C:\Users\sheet\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\sheet\AppData\Local\Temp\ipykernel_2880\3253737092.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  1%|          | 10/920 [19:15<28:16:25, 111.85s/it]

{'loss': 262.168, 'grad_norm': 1241.7386474609375, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  2%|▏         | 20/920 [39:55<35:19:39, 141.31s/it]

{'loss': 266.4482, 'grad_norm': 1818.2353515625, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


...

{'loss': 6.8479, 'grad_norm': 7.218170166015625, 'learning_rate': 1.1904761904761904e-06, 'epoch': 0.99}


100%|██████████| 920/920 [28:10:45<00:00, 94.70s/it]

{'loss': 7.9674, 'grad_norm': 9.574193000793457, 'learning_rate': 0.0, 'epoch': 1.0}


100%|██████████| 920/920 [28:10:48<00:00, 110.27s/it]


{'train_runtime': 101447.9629, 'train_samples_per_second': 0.145, 'train_steps_per_second': 0.009, 'train_loss': 39.04137262261432, 'epoch': 1.0}
